In [2]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
import matplotlib.colors as mcolors
import matplotlib.colors as mcolors
from scipy import stats
import itertools
import mplhep as hep
plt.style.use([hep.style.ROOT, hep.style.firamath])
from glob import glob
from sqlite3_database_plot import *

def parse_list(s):
    return list(map(int, s.strip('[]').split(', ')))


In [2]:
chips = [103,107,111,113,118,122,126]

In [3]:
hexa44 = glob("data_hexa44/chip_*/from_io_delayscan_errorcounts*")
hexa46 = glob("data/chip_113/from_io_delayscan_errorcounts*")


In [32]:
hexa44

['data_hexa44/chip_122/from_io_delayscan_errorcountschip_122.csv',
 'data_hexa44/chip_125/from_io_delayscan_errorcountschip_125.csv',
 'data_hexa44/chip_113/from_io_delayscan_errorcountschip_113.csv',
 'data_hexa44/chip_115/from_io_delayscan_errorcountschip_115.csv',
 'data_hexa44/chip_101/from_io_delayscan_errorcountschip_101.csv',
 'data_hexa44/chip_107/from_io_delayscan_errorcountschip_107.csv',
 'data_hexa44/chip_119/from_io_delayscan_errorcountschip_119.csv',
 'data_hexa44/chip_126/from_io_delayscan_errorcountschip_126.csv',
 'data_hexa44/chip_117/from_io_delayscan_errorcountschip_117.csv',
 'data_hexa44/chip_111/from_io_delayscan_errorcountschip_111.csv',
 'data_hexa44/chip_116/from_io_delayscan_errorcountschip_116.csv',
 'data_hexa44/chip_118/from_io_delayscan_errorcountschip_118.csv',
 'data_hexa44/chip_127/from_io_delayscan_errorcountschip_127.csv',
 'data_hexa44/chip_105/from_io_delayscan_errorcountschip_105.csv',
 'data_hexa44/chip_103/from_io_delayscan_errorcountschip_103.c

In [14]:
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)

def get_max_width(channels, err_counts):
    max_width_by_ch = []
    second_max_width_by_ch = []
    err_wrapped=np.concatenate([err_counts,err_counts[:10]])
    for ch in range(channels):
        x = err_wrapped[:,ch]
        phases = consecutive(np.argwhere(x==0).flatten())
        sizes = [np.size(a) for a in phases]
        max_width = np.max(sizes)
        sizes.remove(max_width)
        try:
            second_max_width = np.max(sizes)
        except:
            second_max_width = 0
        max_width_by_ch.append(max_width)
        second_max_width_by_ch.append(second_max_width)
    return np.array([max_width_by_ch,second_max_width_by_ch])

In [15]:
def phase_scan_eTx(chip=999, hexa=44 ,out_dir='PLOTS',data=glob(f"data_hexa44/chip_{999}/from_io_delayscan_errorcounts*")):
    chip_number = f'{chip}_hexa{hexa}'               
    data = data[0]
    data_eTx = genfromtxt(data,delimiter=',')
    x2 = data_eTx[1:,:]
    max_eTx_, max2_eTx_ = get_max_width(13,x2)
    x2 = np.array(x2)
    x2 = np.minimum(x2, 255)
    norm = mcolors.TwoSlopeNorm(vmin=0, vmax = 255, vcenter=0.1)
    plt.figure(figsize=(15, 10))
    
    a,b=np.meshgrid(np.arange(13),np.arange(63))
    plt.hist2d(a.flatten(),b.flatten(),weights=x2.flatten(),bins=(np.arange(14)-0.5,np.arange(64)-0.5),cmap='bwr',norm=norm);
    plt.xticks(np.arange(0,13,1))
    plt.colorbar().set_label(label='Data transmission errors')
    plt.yticks(np.arange(0,64,4))    
    plt.grid(color='white', linestyle='--', linewidth=.5)
    plt.title(f"ECON-T 1.28 GHz output phase scan {chip_number}")
    plt.ylabel('Phase Select Setting')
    plt.xlabel('Channel Number')
    plt.ylabel('Phase Select Setting')
    plt.xlabel('Channel Number')
    print(f'{chip_number}------------------------')
    print(get_max_width(13,x2))
    odir= out_dir 
    if not os.path.isdir(odir):
        os.makedirs(odir)
    plt.savefig(f'{out_dir}/phase_scan_eTx_chip_{chip_number}_EconT.png',dpi=300,facecolor='w')
    plt.close()
    plt.show()


In [16]:
for chip in chips:
    phase_scan_eTx(chip=chip, hexa=44,out_dir='eTx_plots',data=glob(f"data_hexa44/chip_{chip}/from_io_delayscan_errorcounts*"))
    phase_scan_eTx(chip=chip, hexa=46,out_dir='eTx_plots',data=glob(f"data/chip_{chip}/from_io_delayscan_errorcounts*"))
    
    
    
    
    

103_hexa44------------------------
[[22 20 18 21 18 18 19 17 19 21 19 18 19]
 [17 15 18 18 17 17 18 16 19 16 18 17 17]]
103_hexa46------------------------
[[15 16 26 17 23 25 22 23 22 17 25 25 25]
 [14 15 17 17 17 16 18 16 18 16 17 18 17]]
107_hexa44------------------------
[[16 22 22 23 19 21 18 19 18 23 21 21 23]
 [16 15 17 17 16 16 17 16 17 17 19 18 16]]
107_hexa46------------------------
[[15 15 17 16 23 16 23 22 23 16 25 25 16]
 [14 14 16 16 16 15 17 16 17 15 18 18 16]]
111_hexa44------------------------
[[16 23 21 22 19 21 18 20 19 22 19 19 21]
 [15 17 19 17 18 18 17 19 18 17 17 18 17]]
111_hexa46------------------------
[[15 15 17 16 24 16 22 24 22 16 24 24 16]
 [14 15 15 16 16 16 17 16 18 15 16 18 16]]
113_hexa44------------------------
[[15 21 23 17 21 21 18 22 19 16 21 22 21]
 [15 17 17 17 17 15 17 18 18 15 17 17 16]]
113_hexa46------------------------
[[15 16 16 16 23 16 24 15 24 16 17 17 16]
 [14 15 16 15 16 15 16 15 17 15 16 17 15]]
118_hexa44------------------------
[[16 

In [1]:
def phase_scan_eTx(chip=999, hexa=44 ,out_dir='PLOTS',data=glob(f"data_hexa44/chip_{999}/from_io_delayscan_errorcounts*")):
    norm = mcolors.TwoSlopeNorm(vmin=0, vmax = 255, vcenter=0.1)
    plt.figure(figsize=(15, 10))
    
    a,b=np.meshgrid(np.arange(13),np.arange(63))
    plt.hist2d(a.flatten(),b.flatten(),weights=x2.flatten(),bins=(np.arange(14)-0.5,np.arange(64)-0.5),cmap='bwr',norm=norm);
    plt.xticks(np.arange(0,13,1))
    plt.colorbar().set_label(label='Data transmission errors')
    plt.yticks(np.arange(0,64,4))    
    plt.grid(color='white', linestyle='--', linewidth=.5)
    plt.title(f"ECON-T 1.28 GHz output phase scan {chip_number}")
    plt.ylabel('Phase Select Setting')
    plt.xlabel('Channel Number')
    plt.ylabel('Phase Select Setting')
    plt.xlabel('Channel Number')

NameError: name 'glob' is not defined